# 네이버 카페 크롤러 (Colab)

1. 셀을 순서대로 실행하세요
2. 여러 탭에서 다른 범위로 실행하면 병렬 처리 가능

In [ ]:
# 1. 패키지 설치 (최초 1회)
!pip install playwright playwright-stealth -q
!playwright install chromium
!playwright install-deps

In [ ]:
# 2. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 작업 폴더 생성
!mkdir -p /content/drive/MyDrive/naver_crawler/output

In [ ]:
# 3. auth.json 파일 업로드 (Drive에 이미 있으면 스킵)
import os

auth_in_drive = '/content/drive/MyDrive/naver_crawler/auth.json'
if os.path.exists(auth_in_drive):
    !cp "{auth_in_drive}" /content/auth.json
    print('Drive에서 auth.json 복사 완료!')
else:
    from google.colab import files
    print('auth.json 파일을 업로드하세요:')
    uploaded = files.upload()
    # Drive에도 백업
    !cp auth.json "{auth_in_drive}"
    print('Drive에 백업 완료!')

In [ ]:
#@title 4. 크롤링 설정 { run: "auto" }
#@markdown ### 크롤링 범위 설정
#@markdown 여러 탭에서 다른 범위로 실행하면 병렬 처리됩니다.

START = 0 #@param {type:"integer"}
END = 500 #@param {type:"integer"}
WORKER_ID = 1 #@param {type:"integer"}

#@markdown ---
#@markdown ### 병렬 실행 예시
#@markdown | 탭 | START | END | WORKER |
#@markdown |---|---|---|---|
#@markdown | 1 | 0 | 500 | 1 |
#@markdown | 2 | 500 | 1000 | 2 |
#@markdown | 3 | 1000 | 1500 | 3 |

import os
os.environ['START'] = str(START)
os.environ['END'] = str(END)
os.environ['WORKER'] = str(WORKER_ID)
os.environ['OUTPUT_DIR'] = '/content/drive/MyDrive/naver_crawler/output'

print(f'설정 완료: {START} ~ {END} (Worker {WORKER_ID})')

In [ ]:
# 5. 크롤러 스크립트 업로드 (Drive에 이미 있으면 스킵)
import os

script_in_drive = '/content/drive/MyDrive/naver_crawler/playwright_getIds_auto.py'
if os.path.exists(script_in_drive):
    !cp "{script_in_drive}" /content/playwright_getIds_auto.py
    print('Drive에서 스크립트 복사 완료!')
else:
    from google.colab import files
    print('playwright_getIds_auto.py 파일을 업로드하세요:')
    uploaded = files.upload()
    # Drive에도 백업
    !cp playwright_getIds_auto.py "{script_in_drive}"
    print('Drive에 백업 완료!')

In [ ]:
# 6. 크롤링 실행!
!python playwright_getIds_auto.py

In [ ]:
# 7. (선택) 결과 확인
import pandas as pd
import glob

csv_files = glob.glob('/content/drive/MyDrive/naver_crawler/output/*.csv')
for f in sorted(csv_files):
    print(f'\n=== {os.path.basename(f)} ===')
    df = pd.read_csv(f)
    print(df.tail(5))
    print(f'총 {len(df)}개 행')

In [ ]:
# 8. (선택) 모든 워커 결과 병합
import pandas as pd
import glob

csv_files = glob.glob('/content/drive/MyDrive/naver_crawler/output/*.csv')
if csv_files:
    all_data = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    all_data = all_data.drop_duplicates(subset=['번호']).sort_values('번호')
    
    merged_path = '/content/drive/MyDrive/naver_crawler/output/merged_all.csv'
    all_data.to_csv(merged_path, index=False, encoding='utf-8-sig')
    print(f'병합 완료: {merged_path}')
    print(f'총 {len(all_data)}개 행')